### This notebook will be an initial exploration of Folium's capabilities. 

Per the use cases supplied by ACEP, we want to create a map of Alaska with icons of solar installations over the various regions.  When the cursor hovers over the region, a box will pop up which displays the number of installations that were analyzed in that region along with the highest normalized annual production, and the average normalized annual production.  

In [1]:
import os
import folium
import pandas as pd

In [3]:
alaska_map = folium.Map(location = [63.977161, -152.024667], zoom_start = 4)
alaska_map

In [4]:
alaska_terrain = folium.Map(location = [63.977161, -152.024667], tiles = 'Stamen Terrain', zoom_start = 4)
alaska_terrain

In [6]:
tooltip = 'Click for more info'
#add a marker for the site of Fairbanks. Also, add a hoverover text box.
icon_type = 'star'
folium.Marker([64.838033, -147.668970], popup = 'Fairbanks, AK.', icon = folium.Icon(icon = icon_type),
              tooltip = tooltip).add_to(alaska_terrain)
alaska_terrain

In [9]:
#Add markers for other major cities.
#Anchorage
folium.Marker([61.193625, -149.694974], popup = 'Anchorage, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Kodiak
folium.Marker([57.794934, -152.392466], popup = 'Kodiak, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Nome
folium.Marker([64.516804, -165.399455], popup = 'Nome, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Juneau
folium.Marker([58.306096, -134.422802], popup = 'Juneau, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Prudhoe Bay
folium.Marker([70.212820, -148.408886], popup = 'Prudhoe Bay, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Kotzebue
folium.Marker([66.888948, -162.512280], popup = 'Kotzebue, AK.',
              tooltip = tooltip).add_to(alaska_terrain)
#Bethel
folium.Marker([60.794938, -161.770716], popup = 'Bethel, AK.', 
              tooltip = tooltip).add_to(alaska_terrain)
#Function that enables the map to show a lattitude and longitude when clicked.
alaska_terrain.add_child(folium.LatLngPopup())
#Display the map we've made.
alaska_terrain

You can also embed into each of these markers things like graphs. We would just need three lines of text, which would probably have to be hand-updated. Or, it could reference a variable which is set by a function call? Unclear what the right strategy is now, but we can definitely find an answer here. 

In [8]:
#To save this graphic as an interactive map:
#alaska_terrain.save("Practice Marker Map.html")

For us, the next step then would be to find a way to express data in these popups. I'll just do it for Fairbanks, as right now the S.o.A. Fairbanks panels are the ones we've been working with. 

In [13]:
#First step is to write the code that pulls in the data. We'll work with pre-formatted data. 
panel_data = pd.read_excel(io = 'spirit_of_alaska_fairbanks_cleaned.xlsx')
panel_data.head()

,Year,Month,Johansen Solar (kWh),Gillam Solar (kWh)
0,2011,May,93.412,NaN
1,2011,June,516.681,NaN
2,2011,July,535.737,NaN
3,2011,August,447.844,NaN
4,2011,September,328.926,NaN


In [49]:
#Next, we'll need to generate a representation of the data we want to present as a table, in HTML
#First step: Calculate the normalized annual production. 
normalized_annual_production = panel_data.mean(axis = 0, skipna = True)
print(normalized_annual_production)
print(normalized_annual_production[1])
#Ok, and the last thing is to normalize by the capacity of the Johansen Solar array - 2.8 kW
johansen_normalized = normalized_annual_production[1]/2.8
print(johansen_normalized)

Year                    2014.833333
Johansen Solar (kWh)     288.433209
Gillam Solar (kWh)       207.174182
dtype: float64
288.4332087912088
103.01186028257459


In [50]:
#Ok, so we can also use PVWatts to calculate what a similar system should produce in a year.
#To make it comparable, we'll normalize the predictions based on the installation capacity.
#We'll simulate a 4 kW system, using normal pvwatts parameters and for Fairbanks AK.
#Our results is 3,698 kWh per year for a 4 kW DC system.
pvwatts = 3698
pvwatts_normalized = pvwatts/4

#Next, we'll make an entry for the number of installations: 1
number_of_installations = 1

#And finally, make the dataframe we'll use:
fairbanks_display_dataframe = pd.DataFrame(data = [['Number of Installations', int(number_of_installations)],
                                                  ['PVWatts Predicted Annual Power', pvwatts_normalized],
                                                  ['Average Produced Annual Power', johansen_normalized],
                                                  ['Maximum Produced Annual Power', johansen_normalized]],
                                           columns = ['Local Values', 'Quantity'])
print(fairbanks_display_dataframe)

                     Local Values   Quantity
0         Number of Installations    1.00000
1  PVWatts Predicted Annual Power  924.50000
2   Average Produced Annual Power  103.01186
3   Maximum Produced Annual Power  103.01186


In [51]:
#Ok, and now it's time to embed that dataframe!
html = fairbanks_display_dataframe.to_html()
fairbanks_popup = folium.Popup(html)
folium.Marker([64.838033, -147.668970], popup = fairbanks_popup).add_to(alaska_terrain)
alaska_terrain

In [53]:
#And then save this graphic too. 
#alaska_terrain.save(os.path.join('alaska map.html', 'html_popups.html'))

In [2]:
#Exploring making the map icon interactive:
#Pulled directly offline - 
import folium
from ipywidgets import interact

cities = ["Berlin", "Paris", "Chicago", "Singapore"]
examples = ["Traffic", "Truck info", "Transit", "Regular", "Satellite"]
@interact(city=cities, example=examples)
def show_canned_examples(city, example):
    attr = "HERE.com"
    latlon_for_city = {
        "Berlin": (52.518611, 13.408333), 
        "Paris": (48.8567, 2.3508), 
        "Chicago": (41.88416, -87.63243),
        "Singapore": (1.283333, 103.833333)
    }
    zoom = 14
    queries = {
        "Traffic":
            "https://1.traffic.maps.api.here.com/maptile/2.1/traffictile/newest/normal.traffic.day/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Regular":
            "https://1.base.maps.api.here.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Truck info":
            "https://1.base.maps.api.here.com/maptile/2.1/trucktile/newest/normal.day.grey/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Transit":
            "https://1.base.maps.api.here.com/maptile/2.1/maptile/newest/normal.day.transit/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
        "Satellite":
            "https://1.aerial.maps.api.here.com/maptile/2.1/maptile/newest/hybrid.day/{z}/{x}/{y}/256/png8?lg=eng&app_id=%s&app_code=%s" % (app_id, app_code),
    }
    return folium.Map(location=latlon_for_city[city], tiles=queries[example],attr=attr, zoom_start=zoom)

interactive(children=(Dropdown(description='city', options=('Berlin', 'Paris', 'Chicago', 'Singapore'), value=…

In [4]:
from ipywidgets import interact

cities = ["Berlin", "Paris", "Chicago", "Singapore"]
#examples = ["Traffic", "Truck info", "Transit", "Regular", "Satellite"]
@interact(city=cities)
def show_canned_examples(city):
    attr = "HERE.com"
    latlon_for_city = {
        "Berlin": (52.518611, 13.408333), 
        "Paris": (48.8567, 2.3508), 
        "Chicago": (41.88416, -87.63243),
        "Singapore": (1.283333, 103.833333)
    }
    zoom = 14
    return folium.Map(location = latlon_for_city[city])

interactive(children=(Dropdown(description='city', options=('Berlin', 'Paris', 'Chicago', 'Singapore'), value=…